In [5]:
from utils import *

import numpy as np


# For photon number 1

phot_num = 0

scat_num = 0

max_num_scat = 1000000

escaped_photons = 0

w = 0

while escaped_photons < 1000:

    for i in range(0, max_num_scat):
        print("Starting at w =0, tau = 10**4")

        # Get random direction
        theta,phi = random_theta_phi(phot_num=phot_num,scat_num=scat_num)
        print("Theta, phi = ", theta, phi)

        # Random tau

        tau = random_tau(0,scat_num=scat_num)
        print("Tau = ", tau)

        #get tau_geo
        tau_geo = get_tau_to_surface(theta,phi,tau)

        tau_int = get_tau_int(w)

        

        if tau_int < tau_geo:
            print("Photon has escaped at w = ", w)
            escaped_photons+=1
            break

        else:
            print("Photon has not escaped at w = ", w)
            wp = w
            # Update w
            w = get_wp_from_random_variate(phot_num=0,
                                        scat_num=scat_num,
                                        wc=wp)
            
            print("New w = ", w)

            # Update scat_num 
            scat_num += 1

            print("Scat_num = ", scat_num)

            print("")

        phot_num+=1





Starting at w =0, tau = 10**4
Theta, phi =  1.5084327167000202 5.246344312535077
Tau =  2.8740901269308843
Photon has not escaped at w =  0
New w =  -0.4323563925687264
Scat_num =  1

Starting at w =0, tau = 10**4
Theta, phi =  1.9995436406412057 3.748259497183444
Tau =  0.4453828761606277
Photon has not escaped at w =  -0.4323563925687264
New w =  -0.5301314109009548
Scat_num =  2

Starting at w =0, tau = 10**4
Theta, phi =  2.6218500367824253 1.814981275978436
Tau =  1.5362125995701112
Photon has not escaped at w =  -0.5301314109009548
New w =  -0.5738447568043736
Scat_num =  3

Starting at w =0, tau = 10**4
Theta, phi =  0.576796405523588 0.2698726779061544
Tau =  0.8947205138378502
Photon has not escaped at w =  -0.5738447568043736
New w =  -1.1402091885725356
Scat_num =  4

Starting at w =0, tau = 10**4
Theta, phi =  1.5495039502404917 6.1176509896021924
Tau =  0.348605533945326
Photon has not escaped at w =  -1.1402091885725356
New w =  -0.8741961630567436
Scat_num =  5

Starting

CDF of w' direction is saved as Rwwp_cdf.txt
